In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
import re

# Load Some Data to preprocess Vietnamese languages

In [7]:
# EMOJI
with open('../DATA/files/emojicon.txt', 'r', encoding="utf8") as file:
    emoji_lst = file.read().split('\n')
    emoji_dict = {}
    for line in emoji_lst:
        key, value = line.split('\t')
        emoji_dict[key] = str(value)

# TEEN CODE
with open('../DATA/files/teencode.txt', 'r', encoding="utf8") as file:
    teen_lst = file.read().split('\n')
    teen_dict = {}
    for line in teen_lst:
        key, value = line.split('\t')
        teen_dict[key] = str(value)

# ENG VIET
with open('../DATA/files/english-vnmese.txt', 'r', encoding="utf8") as file:
    eng_lst = file.read().split('\n')
    eng_dict = {}
    for line in eng_lst:
        key, value = line.split('\t')
        eng_dict[key] = str(value)

# WRONG WORD
with open('../DATA/files/wrong-word.txt', 'r', encoding="utf8") as file:
    wrong_lst = file.read().split('\n')

# STOP WORD
with open('../DATA/files/vietnamese-stopwords.txt', 'r', encoding="utf8") as file:
    stop_lst = file.read().split('\n')

# Load file data:

In [2]:
use_cols = ['product_id','sub_category','price','rating','clean_desc','product_name','clean_prd_name']
clean_prd_df = pd.read_csv('../DATA/clean_products.csv',usecols=use_cols)

In [3]:
clean_prd_df.head()

,product_id,product_name,sub_category,price,rating,clean_desc,clean_prd_name
0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Áo Ba Lỗ,86250.0,4.9,áo lỗ chiều đường_phố nhiệt_đới tal fit xuất_x...,áo lỗ thun gân form body
1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Áo Ba Lỗ,26800.0,4.9,áo lỗ xuất_xứ việt_nam tổ_chức trách_nhiệm sản...,áo lỗ nam trắng chất_cotton siêu_mát đẹp
2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Áo Ba Lỗ,39500.0,4.8,áo lỗ thương_hiệu tyasuo chiều áo không_thể đư...,áo lỗ nam tyasuo chất vải co_dãn mát không_xù mềm
3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Áo Ba Lỗ,16500.0,4.8,áo lỗ chất_liệu hàng gửi hà_nội áo lỗ nam mặc ...,áo lỗ hàng việt_nam
4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Áo Ba Lỗ,45000.0,4.8,áo lỗ chiều áo không_thể hàng gửi hà_nội thông...,áo thun nam thể_thao lỗ mẫu mới đẹp


## Check na values from clean data

In [4]:
drop_idx = clean_prd_df[clean_prd_df['clean_prd_name'].apply(lambda x: isinstance(x,float))].index

In [5]:
clean_prd_df = clean_prd_df.drop(index=drop_idx)

In [13]:
#stop_lst

# Cosine_similarity
- link https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
- link https://en.wikipedia.org/wiki/Cosine_similarity

In [8]:
tf_model = TfidfVectorizer(analyzer='word', stop_words=stop_lst)

In [14]:
tfidf_matrix = tf_model.fit_transform(raw_documents=clean_prd_df['clean_prd_name'])

In [15]:
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

MemoryError: Unable to allocate 18.3 GiB for an array with shape (49594, 49594) and data type float64

In [ ]:
cosine_similarities

array([[1.        , 0.69519827, 0.1551208 , 0.12374782, 0.15994914,
        0.00965768, 0.04347849, 0.04908612, 0.05106241],
       [0.69519827, 1.        , 0.15715171, 0.10934188, 0.21851252,
        0.01015654, 0.10185011, 0.06193459, 0.0751564 ],
       [0.1551208 , 0.15715171, 1.        , 0.19403449, 0.26562584,
        0.0304808 , 0.06240797, 0.0773547 , 0.04822855],
       [0.12374782, 0.10934188, 0.19403449, 1.        , 0.1497956 ,
        0.08191914, 0.08858269, 0.0278274 , 0.08692276],
       [0.15994914, 0.21851252, 0.26562584, 0.1497956 , 1.        ,
        0.00771569, 0.13257418, 0.06271941, 0.12209577],
       [0.00965768, 0.01015654, 0.0304808 , 0.08191914, 0.00771569,
        1.        , 0.04378038, 0.00966135, 0.        ],
       [0.04347849, 0.10185011, 0.06240797, 0.08858269, 0.13257418,
        0.04378038, 1.        , 0.06232388, 0.10206116],
       [0.04908612, 0.06193459, 0.0773547 , 0.0278274 , 0.06271941,
        0.00966135, 0.06232388, 1.        , 0.14212572],


In [ ]:
cosine_similarities.shape

(9, 9)

In [ ]:
# với mỗi sản phẩm, lấy 3 sản phẩm tương quan nhất
results = {}

for idx, row in df.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[-4:-1]
    similar_items = [(cosine_similarities[idx][i]) for i in similar_indices]
    similar_items = [(cosine_similarities[idx][i], df.index[i]) for i in similar_indices]
    print(similar_items[0:])
    results[idx] = similar_items[0:]

[(0.15512079719991662, 2), (0.15994914305652225, 4), (0.6951982684968354, 1)]
[(0.15715171135957345, 2), (0.218512522363034, 4), (0.6951982684968354, 0)]
[(0.15715171135957345, 1), (0.19403448802686324, 3), (0.26562584202409356, 4)]
[(0.12374782171555312, 0), (0.14979560104276052, 4), (0.19403448802686324, 2)]
[(0.15994914305652225, 0), (0.218512522363034, 1), (0.26562584202409356, 2)]
[(0.03048080117011501, 2), (0.043780376453166577, 6), (0.08191914396190085, 3)]
[(0.10185010798524011, 1), (0.1020611638551705, 8), (0.13257417789271964, 4)]
[(0.06271940602042296, 4), (0.07735469936250719, 2), (0.14212571768388169, 8)]
[(0.1020611638551705, 6), (0.12209577477360442, 4), (0.14212571768388169, 7)]


In [ ]:
results[0]

[(0.15512079719991662, 2), (0.15994914305652225, 4), (0.6951982684968354, 1)]

In [16]:
def item(id_prd,df):
    text =  df.loc[df['product_id'] == id_prd,'clean_prd_name']
    return text